In [10]:
# Clasificador de árboles con MobileNet: apamate, araguaney y saman
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from PIL import Image


# Configuración de rutas y parámetros
base_dir = 'dataset_llovizna'
clases = ['apamate', 'araguaney', 'saman']
img_size = (224, 224)
batch_size = 16


In [11]:
import os
import shutil
import random

# Configura tus clases y rutas
base_dir = 'dataset_llovizna'
clases = ['apamate', 'araguaney', 'saman']
random.seed(42)

for clase in clases:
    clase_dir = os.path.join(base_dir, clase)
    if not os.path.exists(clase_dir):
        continue
    imgs = [f for f in os.listdir(clase_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    random.shuffle(imgs)
    n_test = int(0.2 * len(imgs))
    test_imgs = imgs[:n_test]
    train_imgs = imgs[n_test:]

    # Crea carpetas destino
    train_dest = os.path.join(base_dir, 'train', clase)
    test_dest = os.path.join(base_dir, 'test', clase)
    os.makedirs(train_dest, exist_ok=True)
    os.makedirs(test_dest, exist_ok=True)

    # Mueve imágenes
    for img in train_imgs:
        shutil.move(os.path.join(clase_dir, img), os.path.join(train_dest, img))
    for img in test_imgs:
        shutil.move(os.path.join(clase_dir, img), os.path.join(test_dest, img))

print("¡Separación de train/test completada!")


¡Separación de train/test completada!


In [12]:
# Preparar generadores de datos
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=img_size,
    batch_size=batch_size,
    classes=clases,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=img_size,
    batch_size=batch_size,
    classes=clases,
    class_mode='categorical',
    subset='validation'
)


Found 0 images belonging to 3 classes.
Found 0 images belonging to 3 classes.
Found 0 images belonging to 3 classes.


In [13]:
# Generadores de datos para la nueva estructura
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    os.path.join(base_dir, 'train'),
    target_size=img_size,
    batch_size=batch_size,
    classes=clases,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    os.path.join(base_dir, 'train'),
    target_size=img_size,
    batch_size=batch_size,
    classes=clases,
    class_mode='categorical',
    subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    os.path.join(base_dir, 'test'),
    target_size=img_size,
    batch_size=batch_size,
    classes=clases,
    class_mode='categorical',
    shuffle=False
)


Found 240 images belonging to 3 classes.
Found 60 images belonging to 3 classes.
Found 74 images belonging to 3 classes.
Found 60 images belonging to 3 classes.
Found 74 images belonging to 3 classes.


In [14]:
# Definir el modelo con MobileNetV2 como base
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=img_size + (3,))
base_model.trainable = False  # Congelar pesos de MobileNetV2

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(len(clases), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,422,339 (9.24 MB)

 Trainable params: 164,355 (642.01 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [15]:
# Entrenar el modelo
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator
)


Epoch 1/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 168ms/step - accuracy: 0.3463 - loss: 1.1955 - val_accuracy: 0.5500 - val_loss: 0.9829
Epoch 2/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 168ms/step - accuracy: 0.3463 - loss: 1.1955 - val_accuracy: 0.5500 - val_loss: 0.9829
Epoch 2/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 110ms/step - accuracy: 0.5610 - loss: 0.8629 - val_accuracy: 0.6833 - val_loss: 0.7337
Epoch 3/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 110ms/step - accuracy: 0.5610 - loss: 0.8629 - val_accuracy: 0.6833 - val_loss: 0.7337
Epoch 3/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 118ms/step - accuracy: 0.7701 - loss: 0.6082 - val_accuracy: 0.7167 - val_loss: 0.6936
Epoch 4/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 118ms/step - accuracy: 0.7701 - loss: 0.6082 - val_accuracy: 0.7167 - val_loss: 0.6936
Epoch 4/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 107ms/step - accuracy: 0.7576 - loss: 0.5663 - val_accuracy: 0.7333 - val_loss: 0.5979
Epoch 5/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 107ms/step - accuracy: 0.7576 - loss: 0.5663 - val_accuracy: 0.

In [16]:
# Evaluar el modelo
val_loss, val_acc = model.evaluate(val_generator)
print(f"Precisión en validación: {val_acc:.2%}")


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.8254 - loss: 0.4476
Precisión en validación: 85.00%
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.8254 - loss: 0.4476
Precisión en validación: 85.00%


In [17]:
# Guardar el modelo entrenado
model.save('mobilenet_tree_classifier.h5')
print('Modelo guardado como mobilenet_tree_classifier.h5')

# Probar el modelo con una imagen externa
from tensorflow.keras.preprocessing import image
import numpy as np

# Cambia la ruta a la imagen que quieras probar
# test_img_path = 'araguaneyPrueba.jpg'
test_img_path = 'samanPrueba.jpg'
img = image.load_img(test_img_path, target_size=img_size)
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0

pred = model.predict(img_array)
print(pred)
clase_predicha = clases[np.argmax(pred)]
print(f'La clase predicha es: {clase_predicha}')


Modelo guardado como mobilenet_tree_classifier.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 327ms/step
[[0.09073075 0.01234016 0.8969291 ]]
La clase predicha es: saman
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 327ms/step
[[0.09073075 0.01234016 0.8969291 ]]
La clase predicha es: saman
